In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

В этом задании вы будете работать с [данными о пациентах, у части которых есть заболевание сердца](https://www.kaggle.com/ronitf/heart-disease-uci). Вам нужно построить модель, классифицирующую пациентов на больных этим заболеванием и тех, у кого его нет, а также проанализировать результаты.

In [2]:
data = pd.read_csv('heart.csv')
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


Разделим данную выборку на обучающую и тестовую части в отношении 3:1.

In [3]:
X_train, X_test, y_train, y_test = train_test_split(data.drop('target', axis=1), 
                                                    data['target'], 
                                                    test_size=0.25, 
                                                    random_state=13)
X_train.shape, X_test.shape

((227, 13), (76, 13))

1. Обучите решающее дерево из `sklearn` (`sklearn.tree.DecisionTreeClassifier`) без ограничения на максимальную глубину (`max_depth=None`). В качестве сида поставьте `random_state=13`.

   Подробнее о параметрах можно почитать в [документации](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html).

   Найдите долю правильных ответов полученного алгоритма на обучающей выборке (**в процентах**). Ответ округлите до двух знаков после запятой. Например, если значение доли правильных ответов будет равно 0.1234, то ответом будет 12.34 - ведь это 12.34%.

In [4]:
clf = DecisionTreeClassifier(random_state=13).fit(X_train, y_train)
y_pred_train = clf.predict(X_train)
accuracy_train = round(accuracy_score(y_train, y_pred_train)*100, 2)
accuracy_train

100.0

In [5]:
# Небольшая проверка для себя, чтобы можно было понять если ошибка именно в этом задании
import hashlib

assert hashlib.sha256(str(round(accuracy_train, 2)).encode()).hexdigest() == '43b87f618caab482ebe4976c92bcd6ad308b48055f1c27b4c574f3e31d7683e0'

In [6]:
# проверка, просто запустите ячейку


2. Теперь найдите долю правильных ответов полученного алгоритма на тестовой выборке (**в процентах**). Ответ округлите до двух знаков после запятой.

    Заметно ли переобучение?

In [7]:
y_pred = clf.predict(X_test)
accuracy_test = round(accuracy_score(y_test, y_pred)*100, 2)
accuracy_test

67.11

In [8]:
# Небольшая проверка для себя, чтобы можно было понять если ошибка именно в этом задании
import hashlib

assert hashlib.sha256(str(round(accuracy_test, 2)).encode()).hexdigest() == '8259d742e25eaf89c66cf0b339c54d6d2ecfbf52711424ffc606a1c9b2da475a'

In [9]:
# проверка, просто запустите ячейку


3. Подберите с помощью кросс-валидации оптимальные гиперпараметры алгоритма. Выбирайте из следующих наборов:


- `max_depth`: [3, 4, 5, 6, 7, 8, 9, 10, None]
- `max_features`: ['auto', 'log2', None]
- `min_samples_leaf`: range(1, 10)
- `min_samples_split`: range(2, 10)
- `criterion`: ['gini', 'entropy']

    В этом вам поможет метод `sklearn.model_selection.GridSearchCV`. Подробнее о том, какие параметры и методы в нем используются, и о примерах работы с ним можно прочитать в [документации](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).
    
    1) Создайте решающее дерево - не забудьте поставить `random_state=13`.
    
    2) Задайте `param_grid` - сетку (словарь) гиперпараметров и их значений, по которой будет проходить метод.
    
    3) Вызовите метод `GridSearchCV` - в качестве параметра `estimator` задайте решающее дерево из первого шага, `param_grid` - сетку из второго. Задайте параметр `cv=5`, чтобы кросс-валидация проходила по 5 фолдам. Также задайте параметр `scoring='accuracy'`, чтобы оценка качества моделей на кросс-валидации проходила с помощью доли правильных ответов. Запустите метод на обучающей выборке с помощью `fit`. 
    
    4) Выведите лучшие параметры с помощью атрибута `best_params_`.
    
    5) Сохраните оптимальное значение глубины дерева в переменную optimal_depth.
    
    6) Сохраните `best_params_`, закомментируйте вызов GridSearchCV для сдачи (!).
    
    7) Обучите модель с выбраными лучшими параметрами (из `best_params_`) для выполнения последующих заданий.
    
    Какое значение глубины дерева получилось оптимальным?
    
Обращаем ваше внимание на то, что из-за ограничений ноутбука на платформе, при сдаче вам потребуется закомментировать код с методом GridSearchCV и оставитьобучение модели с лучшими параметрами в последующих заданиях.

In [13]:
dtc0 = DecisionTreeClassifier(random_state=13)
param_grid = {'max_depth':[3, 4, 5, 6, 7, 8, 9, 10, None], 
              'max_features': ['auto', 'log2', None],
              'min_samples_leaf': range(1, 10),
              'min_samples_split': range(2, 10),
              'criterion': ['gini', 'entropy'] }

clf = GridSearchCV(dtc0, param_grid, scoring='accuracy', cv=5).fit(X_train, y_train)
clf.best_params_

{'criterion': 'gini',
 'max_depth': 9,
 'max_features': 'auto',
 'min_samples_leaf': 3,
 'min_samples_split': 9}

In [14]:
optimal_depth = 9
dtc1 = DecisionTreeClassifier(random_state=13, 
                             criterion='gini',
                             max_depth=9,
                             max_features= 'auto',
                             min_samples_leaf= 3,
                             min_samples_split=9).fit(X_train, y_train)

**Напоминание:** при сдаче ноутбука оставляйте только обучение модели с лучшими параметрами, а вызов GridSearchCV закомментируйте для верной работы грейдера.

In [15]:
# Небольшая проверка для себя, чтобы можно было понять если ошибка именно в этом задании
import hashlib

assert hashlib.sha256(str(optimal_depth).encode()).hexdigest() == '19581e27de7ced00ff1ce50b2047e7a567c76b1cbaebabe5ef03f7c3017bb5b7'

In [16]:
# проверка, просто запустите ячейку


4. Обучите модель с найденными в прошлом задании наилучшими параметрами (из `best_params_`). Эту модель используйте для всех последующих заданий. Найдите долю правильных ответов решающего дерева с подобранными оптимальными значениями гиперпараметров на обучающей выборке (**в процентах**). Ответ округлите до двух знаков после запятой.



In [17]:
y_pred = dtc1.predict(X_train)
optim_accuracy_train = round(accuracy_score(y_train, y_pred)*100, 2)
optim_accuracy_train

90.75

In [18]:
# Небольшая проверка для себя, чтобы можно было понять если ошибка именно в этом задании
import hashlib

assert hashlib.sha256(str(round(optim_accuracy_train, 2)).encode()).hexdigest() == 'af3ae375cd766392c55f07224f20efe7297f45f8aa3b6d4a245dafe7766e8938'

In [19]:
# проверка, просто запустите ячейку


5. Найдите долю правильных ответов решающего дерева с подобранными оптимальными значениями гиперпараметров на тестовой выборке (**в процентах**). Ответ округлите до двух знаков после запятой.

    Уменьшилось ли переобучение?

In [20]:
y_pred_test = dtc1.predict(X_test)
optim_accuracy_test = round(accuracy_score(y_test, y_pred_test)*100, 2)
optim_accuracy_test

73.68

In [21]:
# Небольшая проверка для себя, чтобы можно было понять если ошибка именно в этом задании
import hashlib

assert hashlib.sha256(str(round(optim_accuracy_test, 2)).encode()).hexdigest() == '1b5b7519046fa5a1b6f34e5af1991f12f16063ea5b4389e83a0bd16991ab29a5'

In [22]:
# проверка, просто запустите ячейку


6. Решающее дерево позволяет предсказывать не только классы, но и вероятности классов - с помощью метода `predict_proba`. Посмотрите на вероятности классов полученного решающего дерева и посчитайте значение AUC-ROC. Ответ округлите до двух знаков после запятой.

In [23]:
roc_auc = round(roc_auc_score(y_test, dtc1.predict_proba(X_test)[:, 1]),2)
roc_auc

0.77

In [24]:
# Небольшая проверка для себя, чтобы можно было понять если ошибка именно в этом задании
import hashlib

assert hashlib.sha256(str(round(roc_auc, 2)).encode()).hexdigest() == '91c20a20ec3b329a3cde25fc4cc3d347416be779be540d93902774d885eccd94'

In [25]:
# проверка, просто запустите ячейку


7. Какой признак является самым важным по мнению полученного решающего дерева? Чтобы это проверить, вам поможет атрибут `feature_importances_`.

In [26]:
importances = dtc1.feature_importances_
list_columns = X_train.columns.values.tolist()
dtc_importances = pd.Series(importances, index=list_columns).sort_values(ascending=False)
best_feature = dtc_importances.index[0]
best_feature 

'slope'

In [27]:
# проверка, просто запустите ячейку
